In [1]:
import torch

# Automatically choose GPU if available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")


Using device: cuda


In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import pandas as pd

In [3]:
df = pd.read_csv("Datasets/flipkart_product.csv",encoding='ISO-8859-1')
embeddings = OllamaEmbeddings(model="llama3.2")

In [4]:
df.head()

,ProductName,Price,Rate,Review,Summary
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,Great cooler.. excellent air flow and for this...
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,Best budget 2 fit cooler. Nice cooling
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,The quality is good but the power of air is de...
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,Very bad product it's a only a fan
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,Ok ok product


In [5]:
df["ProductName"].unique()

array(['Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)',
       'Candes 60 L Room/Personal Air Cooler?ÿ?ÿ(White, Black, CRETA)',
       'MAHARAJA WHITELINE 65 L Desert Air Cooler?ÿ?ÿ(White, Grey, Rambo Grey / AC-303)',
       'Crompton 75 L Desert Air Cooler?ÿ?ÿ(White, Teal, ACGC-DAC751)',
       'colcum Collapsible Wardrobe 88130 Micro Fiber Collapsible WardrobeÂ\xa0Â\xa0(Finish Color - Maroon, DIY(Do-It-Yourself))',
       'FLIPZON Baby 6 Shelve Unbreakable Material PVC Collapsible WardrobeÂ\xa0Â\xa0(Finish Color - Blue, DIY(Do-It-Yourself))',
       'S . K Store Carbon Steel Collapsible WardrobeÂ\xa0Â\xa0(Finish Color - Black, DIY(Do-It-Yourself))',
       'Furnifry Wooden TV Entertainment Unit/Wall Set Top Box Shelf Stand/TV Unit Bedroom/ Engineered Wood TV Entertainment UnitÂ\xa0Â\xa0(Finish Color - Black and Walnut, DIY(Do-It-Yourself))',
       'Zofiy 6 Layer Metal Frame Clothes Organizer - Maroon PP Collap

In [6]:
db_location = "./chrome_langchain_db"
add_documents = not os.path.exists(db_location)

In [7]:
if add_documents:
    documents = []
    ids = []
    
    for i, row in df.iterrows():
        document = Document(
            page_content=str(row["ProductName"]) + " " + str(row["Review"])+" "+str(row["Summary"]),
            metadata={"price": str(row["Price"]), "rate": str(row["Rate"])},
            id=str(i)
        )
        ids.append(str(i))
        documents.append(document)

In [8]:
vector_store = Chroma(
    collection_name="product_reviews",
    persist_directory=db_location,
    embedding_function=embeddings
)

In [9]:
if add_documents:
    vector_store.add_documents(documents=documents, ids=ids)

In [10]:
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5})

In [11]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
#from vector import retriever

model = OllamaLLM(model="llama3.2")

In [12]:
template = """
You are an expert in answering questions about a shopping website that sells different products.
You have to give your response.

Here are some relevant reviews: {reviews}

Here is the question to answer: {question}
"""

In [13]:
prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = prompt | model

In [15]:
while True:
    print("\n\n-------------------------------")
    question = input("Ask your question (q to quit): ")
    print("\n\n")
    if question == "q":
        break
    
    reviews = retriever.invoke(question)
    result = chain.invoke({"reviews": reviews, "question": question})
    print(result)



-------------------------------
Ask your question (q to quit): what kind of tv's do you have?



We offer a wide range of TVs from top brands like Samsung, LG, Sony, and more. Our current lineup includes:

1. Smart TVs with 4K resolution, HDR, and various smart features.
2. LED/LCD TVs with high brightness and color accuracy.
3. OLED TVs for an immersive viewing experience with deep blacks and vibrant colors.

Some popular models we have available include:

- Samsung QLED 8K TVs
- LG NanoCell 4K TVs
- Sony X90J 4K TV
- Vizio P-Series Quantum X 4K TVs

You can browse our full selection of TVs by visiting our website or contacting our customer support team for more information on specific models and features.


-------------------------------
Ask your question (q to quit): q



